In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import csv
import time

chromedriver_path = 'C:/Users/Amal Maatoug/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe'

service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service)

url = 'https://fr.shopping.rakuten.com/nav/Tel-PDA_Telephones-mobiles'

driver.get(url)

data = []

# Fonction pour extraire les données de la page actuelle
def extract_data():
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    selector = '.mh-16.productList_childrenLayout_d73'
    containers = soup.select(selector)
    
    for product in containers:
        title_tag = product.find('p', class_='description_styleTitle_KPO lh-title normal mv0 b di')
        neuf_price_tag = product.find('span', class_='f20 dib b')
        occasion_price_tag = product.find('span', class_='b dib f20')
        description_tag = product.find('ul', class_='description_bulletPoints_qgD')

        title = title_tag.text.strip() if title_tag else 'N/A'
        neuf_price = neuf_price_tag.text.strip() if neuf_price_tag else 'N/A'
        occasion_price = occasion_price_tag.text.strip() if occasion_price_tag else 'N/A'
        description = description_tag.text.strip() if description_tag else 'N/A'

        data.append({
            'Article': title,
            'neuf_price': neuf_price,
            'occasion_price': occasion_price,
            'description': description
        })

try:
    # Ouvrir un fichier CSV pour écrire les données
    with open('RakutenSmartphones.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=['Article', 'neuf_price', 'occasion_price', 'description'])
        writer.writeheader()

        # Boucle pour naviguer sur plusieurs pages
        for page in range(1, 31):  # Assuming we want to scrape the first 30 pages
            try:
                # Attendre que les éléments soient présents
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '.mh-16.productList_childrenLayout_d73')))

                # Extraire les données de la page actuelle
                extract_data()

                # Vérifier si le bouton 'Suivant' est présent et cliquable
                next_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, '//div[@data-qa="nextButton"]/a'))
                )
                next_button.click()
                time.sleep(3)  # Attendre quelques secondes pour que la page se charge complètement

            except Exception as e:
                print("Erreur lors de la navigation ou du scrapping:", e)
                print("Attente de 10 secondes pour intervention manuelle...")
                time.sleep(10)  # Attendre 10 secondes pour intervention manuelle
                continue

        # Écrire les données dans le fichier CSV
        writer.writerows(data)

        print(f"Les données ont été écrites dans RakutenSmartphones.csv")

finally:
    # Fermer le driver Selenium
    driver.quit()


Erreur lors de la navigation ou du scrapping: Message: 

Attente de 10 secondes pour intervention manuelle...
Les données ont été écrites dans RakutenSmartphones.csv
